In [2]:
import numpy as np
import pandas as pd
import torch
import wandb
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from torch import optim, nn
import torchmetrics
from tqdm import tqdm
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger, WandbLogger, CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")


PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [1]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'input_size': {
            'values': [223]
        },
        'hidden_size': {
            'values': [128, 256, 512, 1024]
        },
        'num_layers': {
            'min': 2,
            'max': 15
        },
        'dropout_prob': {
            'values': [0.2, 0.3, 0.4, 0.5]
        },
        'activation': {
            'values': ['ReLU']
        },
        'decrease_size': {
            'values': [False, True]
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'lr': {
            'min': 1e-5,
            'max': 1e-1
        },
        'weight_decay': {
            'values': [1e-5]
        },
        'max_epochs': {
            'values': [200]
        },
        'patience': {
            'values': [10, 20, 30, 40]
        },
    }
}

In [30]:
wandb.login()
wandb_logger = WandbLogger(project='leaguify', log_model='all')
tb_logger = TensorBoardLogger('lightning_logs')
csv_logger = CSVLogger('logs', name='leaguify')

In [31]:
training_data = wandb.Artifact('merged_training_data', type='dataset', )

In [32]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, output_size=1, activation=nn.ReLU(),
                 decrease_size=False):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential()
        self.linear_relu_stack.append(nn.Linear(input_size, hidden_size))
        for i in range(num_layers - 1):
            if decrease_size:
                next_hidden_size = int(self.hidden_size // 2)
            else:
                next_hidden_size = self.hidden_size
            self.linear_relu_stack.append(self.dropout)
            self.linear_relu_stack.append(nn.BatchNorm1d(self.hidden_size))
            self.linear_relu_stack.append(nn.Linear(self.hidden_size, next_hidden_size))
            self.linear_relu_stack.append(activation)
            self.hidden_size = next_hidden_size
        self.linear_relu_stack.append(nn.Linear(self.hidden_size, self.output_size))
        self.linear_relu_stack.append(nn.Sigmoid())

    def forward(self, x):
        """
        
        :param x: 
        :return: 
        """
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [33]:
class LNN(L.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, output_size=1, activation=nn.ReLU(),
                 decrease_size=False):
        super().__init__()
        self.model = NeuralNetwork(input_size, hidden_size, num_layers, dropout_prob, output_size, activation,
                                   decrease_size)
        self.criterion = nn.BCELoss()
        self.save_hyperparameters()
        self.accuracy = torchmetrics.classification.BinaryAccuracy()
        self.f1 = torchmetrics.classification.BinaryF1Score()
        self.confusion_matrix = torchmetrics.classification.BinaryConfusionMatrix()

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        y_hat = self.model(x).squeeze(-1)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', self.accuracy(y_hat, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        y_hat = self.model(x).squeeze(-1)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.accuracy(y_hat, y), prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        y_hat = self.model(x).squeeze(-1)
        print(f'y_shape: {y.shape}, y_hat_shape: {y_hat.shape}')
        loss = self.criterion(y_hat, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', self.accuracy(y_hat, y), prog_bar=True)
        self.log('test_f1', self.f1(y_hat, y), prog_bar=True)
        print(f'test_confusion_matrix {self.confusion_matrix(y_hat, y)}')
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [34]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, )
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64)
        self.transform = transform
        self.target_transform = target_transform
        print(f'labels: {self.labels}')
        self.print_statistics()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

    def print_statistics(self):
        print(f'Number of samples: {len(self.data)}')
        print(f'Number of features: {len(self.data[0])}')
        print(f'Number of labels: {len(self.labels)}')
        print(f'Number of classes: {len(np.unique(self.labels.cpu().numpy()))}')
        print(f'Number of samples per class: {np.bincount(self.labels.cpu().numpy())}')

In [35]:
def get_train_data():
    """
    :return: 
    """
    return StaticDataset('../data/static_05_12_23/processed/train_static_merged.npy')

In [36]:
def get_val_data():
    """
    
    :return: 
    """
    return StaticDataset('../data/static_05_12_23/processed/val_static_merged.npy')

In [37]:
def get_test_data():
    """
    :return: 
    """
    return StaticDataset('../data/static_05_12_23/processed/test_static_merged.npy')

In [38]:
def make_loader(dataset, batch_size=64, shuffle=True):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)

In [39]:
def make_test_loader(dataset, shuffle=False):
    batch_size = len(dataset)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)

In [40]:
train_loader = make_loader(get_train_data())
val_loader = make_loader(get_val_data(), shuffle=False)
test_loader = make_test_loader(get_test_data(), shuffle=False)

labels: tensor([1, 0, 1,  ..., 0, 0, 0])
Number of samples: 20633
Number of features: 224
Number of labels: 20633
Number of classes: 2
Number of samples per class: [ 9888 10745]
labels: tensor([1, 1, 1,  ..., 1, 1, 1])
Number of samples: 2293
Number of features: 224
Number of labels: 2293
Number of classes: 2
Number of samples per class: [1089 1204]
labels: tensor([1, 1, 0,  ..., 1, 1, 1])
Number of samples: 2548
Number of features: 224
Number of labels: 2548
Number of classes: 2
Number of samples per class: [1170 1378]


In [41]:
model = LNN(input_size=223, hidden_size=512, num_layers=8, dropout_prob=0.2, output_size=1, activation=nn.ReLU(),
            decrease_size=False)
wandb_logger.watch(model)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [3]:
checkpoint_callback = ModelCheckpoint(monitor='val_acc', mode='max', every_n_epochs=1)

In [ ]:
trainer = L.Trainer(max_epochs=100, accelerator=device,
                    logger=[wandb_logger, tb_logger, csv_logger], callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                  | Params
-----------------------------------------------------------
0 | model            | NeuralNetwork         | 2.0 M 
1 | criterion        | BCELoss               | 0     
2 | accuracy         | BinaryAccuracy        | 0     
3 | f1               | BinaryF1Score         | 0     
4 | confusion_matrix | BinaryConfusionMatrix | 0     
-----------------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
7.844     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

y_shape: torch.Size([2548]), y_hat_shape: torch.Size([2548])
test_confusion_matrix tensor([[739, 431],
        [475, 903]], device='cuda:0')
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6444270014762878
         test_f1            0.6659291982650757
        test_loss           0.8485732078552246
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
